In [1]:
import requests as r

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt 
#plt.style.use('fivethirtyeight')

Import the original data file first.

Create a copy of the data file to work on.

In [3]:
#pd.set_option('display.max_columns', None)

df = pd.read_csv('dataset/steam.csv')

# work on this copy for cleaning data
cleaned_df = df.copy()

cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   appid             27075 non-null  int64  
 1   name              27075 non-null  object 
 2   release_date      27075 non-null  object 
 3   english           27075 non-null  int64  
 4   developer         27074 non-null  object 
 5   publisher         27061 non-null  object 
 6   platforms         27075 non-null  object 
 7   required_age      27075 non-null  int64  
 8   categories        27075 non-null  object 
 9   genres            27075 non-null  object 
 10  steamspy_tags     27075 non-null  object 
 11  achievements      27075 non-null  int64  
 12  positive_ratings  27075 non-null  int64  
 13  negative_ratings  27075 non-null  int64  
 14  average_playtime  27075 non-null  int64  
 15  median_playtime   27075 non-null  int64  
 16  owners            27075 non-null  object

Drop columns not relevant.

In [4]:
# Delete rows where irrelavent
cleaned_df = cleaned_df.drop(columns=["appid", "developer", "publisher", "required_age", "steamspy_tags", "platforms"])

cleaned_df.head()

,name,release_date,english,categories,genres,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,Counter-Strike,2000-11-01,1,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,124534,3339,17612,317,10000000-20000000,7.19
1,Team Fortress Classic,1999-04-01,1,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,3318,633,277,62,5000000-10000000,3.99
2,Day of Defeat,2003-05-01,1,Multi-player;Valve Anti-Cheat enabled,Action,0,3416,398,187,34,5000000-10000000,3.99
3,Deathmatch Classic,2001-06-01,1,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,1273,267,258,184,5000000-10000000,3.99
4,Half-Life: Opposing Force,1999-11-01,1,Single-player;Multi-player;Valve Anti-Cheat en...,Action,0,5250,288,624,415,5000000-10000000,3.99


<insert description>

In [5]:
for i in ["categories"]:
    possible = cleaned_df[i].str.split(";").explode().unique()
    for p in possible:
        cleaned_df[i + "_" + p] = cleaned_df[i].str.contains(p, regex=False).astype(int)
    cleaned_df = cleaned_df.drop(columns=[i])

cleaned_df.head()

,name,release_date,english,genres,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,...,categories_Steam Leaderboards,categories_SteamVR Collectibles,categories_Online Co-op,categories_Shared/Split Screen,categories_Local Co-op,categories_MMO,categories_VR Support,categories_Mods,categories_Mods (require HL2),categories_Steam Turn Notifications
0,Counter-Strike,2000-11-01,1,Action,0,124534,3339,17612,317,10000000-20000000,...,0,0,0,0,0,0,0,0,0,0
1,Team Fortress Classic,1999-04-01,1,Action,0,3318,633,277,62,5000000-10000000,...,0,0,0,0,0,0,0,0,0,0
2,Day of Defeat,2003-05-01,1,Action,0,3416,398,187,34,5000000-10000000,...,0,0,0,0,0,0,0,0,0,0
3,Deathmatch Classic,2001-06-01,1,Action,0,1273,267,258,184,5000000-10000000,...,0,0,0,0,0,0,0,0,0,0
4,Half-Life: Opposing Force,1999-11-01,1,Action,0,5250,288,624,415,5000000-10000000,...,0,0,0,0,0,0,0,0,0,0


Drop non-English games. This make it easier to do natural language processing on the dataset later.

In [6]:
print("Dataset dimension before drop: ", cleaned_df.shape)

Dataset dimension before drop:  (27075, 40)


As you can see there are 27075 rows for both English and non-English games.

In [7]:
cleaned_df = cleaned_df[(cleaned_df["english"] == 1)]
print("Dataset dimension after remove non-English games: ", cleaned_df.shape)


Dataset dimension after remove non-English games:  (26564, 40)


As you can see there are 26564 rows for English games only.

Since now all games are English, can drop the english column because it is unnecessary.

In [8]:
cleaned_df = cleaned_df.drop(columns="english")

In [9]:
print("Dataset dimension after remove non-English games: ", cleaned_df.shape)

Dataset dimension after remove non-English games:  (26564, 39)


From 40 columns, now left 39 columns after removing english column. Also shown in below cleaned_df.info(), no more english column.

In [10]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26564 entries, 0 to 27074
Data columns (total 39 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   name                                   26564 non-null  object 
 1   release_date                           26564 non-null  object 
 2   genres                                 26564 non-null  object 
 3   achievements                           26564 non-null  int64  
 4   positive_ratings                       26564 non-null  int64  
 5   negative_ratings                       26564 non-null  int64  
 6   average_playtime                       26564 non-null  int64  
 7   median_playtime                        26564 non-null  int64  
 8   owners                                 26564 non-null  object 
 9   price                                  26564 non-null  float64
 10  categories_Multi-player                26564 non-null  int32  
 11  categor

In [11]:
cleaned_df.head(n=1000)

,name,release_date,genres,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,...,categories_Steam Leaderboards,categories_SteamVR Collectibles,categories_Online Co-op,categories_Shared/Split Screen,categories_Local Co-op,categories_MMO,categories_VR Support,categories_Mods,categories_Mods (require HL2),categories_Steam Turn Notifications
0,Counter-Strike,2000-11-01,Action,0,124534,3339,17612,317,10000000-20000000,7.19,...,0,0,0,0,0,0,0,0,0,0
1,Team Fortress Classic,1999-04-01,Action,0,3318,633,277,62,5000000-10000000,3.99,...,0,0,0,0,0,0,0,0,0,0
2,Day of Defeat,2003-05-01,Action,0,3416,398,187,34,5000000-10000000,3.99,...,0,0,0,0,0,0,0,0,0,0
3,Deathmatch Classic,2001-06-01,Action,0,1273,267,258,184,5000000-10000000,3.99,...,0,0,0,0,0,0,0,0,0,0
4,Half-Life: Opposing Force,1999-11-01,Action,0,5250,288,624,415,5000000-10000000,3.99,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,ARMA: Gold Edition,2011-07-13,Action;Simulation;Strategy,0,548,209,22,22,500000-1000000,5.99,...,0,0,0,0,0,0,0,0,0,0
996,ARMA: Cold War Assault,2011-08-08,Action;Simulation;Strategy,0,3195,942,28,39,1000000-2000000,3.49,...,0,0,0,0,0,0,0,0,0,0
997,Dungeon Defenders,2011-10-18,Action;Indie;RPG;Strategy,114,10693,932,1993,1205,1000000-2000000,9.99,...,1,0,0,1,0,0,0,0,0,0
998,The Bureau: XCOM Declassified,2013-08-22,Action,41,6661,2923,201,168,2000000-5000000,14.99,...,0,0,0,0,0,0,0,0,0,0


We use the rough estimate fr the "owners" category as we only have the range, we use the average 

In [13]:
owner = cleaned_df["owners"].str.split("-")
owner = owner.apply(lambda x: (int(x[0]) + int(x[1]))/2)
owner = owner.astype(int)
cleaned_df["owners"] = owner

Release date is updated to be measured in terms of days until 2024 Jan 1.
2024 Jan 1 has been arbitrarily chosen but it should not affect anything in the long run and is after all the dates in the dataset.

In [18]:
cleaned_df["age"] = (pd.Timestamp("2024-01-01") - pd.to_datetime(cleaned_df["release_date"])).dt.days
cleaned_df = cleaned_df.drop(columns=["release_date"])
cleaned_df.head()

,name,genres,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,categories_Multi-player,...,categories_SteamVR Collectibles,categories_Online Co-op,categories_Shared/Split Screen,categories_Local Co-op,categories_MMO,categories_VR Support,categories_Mods,categories_Mods (require HL2),categories_Steam Turn Notifications,age
0,Counter-Strike,Action,0,124534,3339,17612,317,15000000,7.19,1,...,0,0,0,0,0,0,0,0,0,8461
1,Team Fortress Classic,Action,0,3318,633,277,62,7500000,3.99,1,...,0,0,0,0,0,0,0,0,0,9041
2,Day of Defeat,Action,0,3416,398,187,34,7500000,3.99,1,...,0,0,0,0,0,0,0,0,0,7550
3,Deathmatch Classic,Action,0,1273,267,258,184,7500000,3.99,1,...,0,0,0,0,0,0,0,0,0,8249
4,Half-Life: Opposing Force,Action,0,5250,288,624,415,7500000,3.99,1,...,0,0,0,0,0,0,0,0,0,8827


# Heres what the cleaned dataset looks like

In [19]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26564 entries, 0 to 27074
Data columns (total 39 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   name                                   26564 non-null  object 
 1   genres                                 26564 non-null  object 
 2   achievements                           26564 non-null  int64  
 3   positive_ratings                       26564 non-null  int64  
 4   negative_ratings                       26564 non-null  int64  
 5   average_playtime                       26564 non-null  int64  
 6   median_playtime                        26564 non-null  int64  
 7   owners                                 26564 non-null  int32  
 8   price                                  26564 non-null  float64
 9   categories_Multi-player                26564 non-null  int32  
 10  categories_Online Multi-Player         26564 non-null  int32  
 11  categor

In [20]:
cleaned_df.head()

,name,genres,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,categories_Multi-player,...,categories_SteamVR Collectibles,categories_Online Co-op,categories_Shared/Split Screen,categories_Local Co-op,categories_MMO,categories_VR Support,categories_Mods,categories_Mods (require HL2),categories_Steam Turn Notifications,age
0,Counter-Strike,Action,0,124534,3339,17612,317,15000000,7.19,1,...,0,0,0,0,0,0,0,0,0,8461
1,Team Fortress Classic,Action,0,3318,633,277,62,7500000,3.99,1,...,0,0,0,0,0,0,0,0,0,9041
2,Day of Defeat,Action,0,3416,398,187,34,7500000,3.99,1,...,0,0,0,0,0,0,0,0,0,7550
3,Deathmatch Classic,Action,0,1273,267,258,184,7500000,3.99,1,...,0,0,0,0,0,0,0,0,0,8249
4,Half-Life: Opposing Force,Action,0,5250,288,624,415,7500000,3.99,1,...,0,0,0,0,0,0,0,0,0,8827


Export the cleaned dataset to a CSV file.

In [21]:
cleaned_df.to_csv('steam_cleaned.csv')